In [ ]:
# 初始化库
from canlib import canlib, Frame
import struct

import time


In [ ]:
# 枚举端口
num_channels = canlib.getNumberOfChannels()
print(f"Found {num_channels} channels")
for ch in range(num_channels):
    chd = canlib.ChannelData(ch)
    print(f"{ch}. {chd.channel_name} ({chd.card_upc_no} / {chd.card_serial_no})")

#比特率
cdbBitrate=canlib.Bitrate.BITRATE_1M

# LOG TRIG 功能

In [ ]:
# 建立变量字典
varDict={}

tiMapFileLoc = """C:/Users/t/workspace_v12/P21kw_v2_SiC_S/build_release/P21kw_v2_SiC_S.map"""

with open(tiMapFileLoc, 'r') as f:
  mapFileTxt = f.read()

startMapChaStr = """GLOBAL SYMBOLS: SORTED BY Symbol Address"""

mapFileTxt = mapFileTxt[mapFileTxt.find(startMapChaStr)+len(startMapChaStr):-1]

mapFileTxtLines=mapFileTxt.split('\n')

# 扔掉前面4行和后面3行
mapFileTxtLines=mapFileTxtLines[4:-2]

# 创建变量表字典
for metaTxt in mapFileTxtLines:
    metaTxtList=metaTxt.split()
    if(not metaTxtList[2].startswith("_")):
        continue
    if(metaTxtList[2][1:].startswith("_")):
        continue
    metaTxtList[2]=metaTxtList[2][1:]
    varAddr=int(metaTxtList[1],16)
    if(varAddr < 0xb000 or varAddr >= 0x1C000):
        continue
    # print(metaTxtList[2], hex(varAddr))
    varDict[metaTxtList[2]] = varAddr

# 人工标注非float型的变量
varTypeDict = {}
varTypeDict["CH1_cur_mode"] = "int16_t"
varTypeDict["CH2_cur_mode"] = "int16_t"
varTypeDict["speed_mode"] = "int16_t"
varTypeDict["channel_mode"] = "int16_t"
varTypeDict["CH1_angle_mode"] = "int16_t"
varTypeDict["CH2_angle_mode"] = "int16_t"
varTypeDict["thetaRDC_raw_offset_CH1"] = "uint16_t"
varTypeDict["thetaRDC_raw_offset_CH2"] = "uint16_t"
varTypeDict["thetaRDC_raw"] = "uint16_t"
varTypeDict["IProtectFlg_CH1"] = "int16_t"
varTypeDict["IProtectFlg_CH2"] = "int16_t"
varTypeDict["SPDProtectFlg"] = "int16_t"
varTypeDict["UdcProtectFlg_CH1"] = "int16_t"
varTypeDict["UdcProtectFlg_CH2"] = "int16_t"
varTypeDict["tempProtectFlg"] = "int16_t"
varTypeDict["isr_start_pwm_cnt"] = "uint16_t"
varTypeDict["isr_end_pwm_cnt"] = "uint16_t"
varTypeDict["targetWaveMode"] = "int16_t"
varTypeDict["CH1_Iv_raw"] = "int16_t"
varTypeDict["CH1_Iw_raw"] = "int16_t"
varTypeDict["CH1_Idc_raw"] = "int16_t"
varTypeDict["CH1_Iv_raw_offset"] = "int16_t"
varTypeDict["CH1_Iw_raw_offset"] = "int16_t"
varTypeDict["CH1_Idc_raw_offset"] = "int16_t"
varTypeDict["CH2_Iu_raw"] = "int16_t"
varTypeDict["CH2_Iv_raw"] = "int16_t"
varTypeDict["CH2_Idc_raw"] = "int16_t"
varTypeDict["CH2_Iu_raw_offset"] = "int16_t"
varTypeDict["CH2_Iv_raw_offset"] = "int16_t"
varTypeDict["CH2_Idc_raw_offset"] = "int16_t"
varTypeDict["ad2s1210ErrGPIO"] = "uint16_t"
varTypeDict["dbg_ad2s1210Err"] = "uint16_t"
varTypeDict["ad2s1210WaitCnt"] = "int16_t"
varTypeDict["ADRC_mode"] = "uint16_t"
varTypeDict["RDV_err_check"] = "function"
varTypeDict["adcOffset_init"] = "function"
varTypeDict["scd_Udc_set"] = "function"
varTypeDict["sram_init"] = "function"
varTypeDict["ad2s1210_ClaAccModeEn"] = "function"

In [ ]:
# 解析结构体, 暂时只对纯float结构体
stru2Dtab_PIctrl_struct = [
    ["kp", 0],
    ["ki", 2],
    ["kb", 4],
    ["max", 6],
    ["min", 8],
    ["integral", 10],
    ["ans", 12],
]

stru2Dtab_Trans_struct = [
    ["U", 0],
    ["V", 2],
    ["W", 4],
    ["al", 6],
    ["be", 8],
    ["d", 10],
    ["q", 12],
    ["abdq0", 14],
]

def addStructVarUtil(structName, struInfoTab):
    global varDict
    for varInStru in struInfoTab:
        varDict[structName+"."+varInStru[0]] = varDict[structName]+varInStru[1]

addStructVarUtil("UdcPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("omegaPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("CH1_IdPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("CH1_IqPI", stru2Dtab_PIctrl_struct)

addStructVarUtil("CH1_ifbk", stru2Dtab_Trans_struct)



In [ ]:
# 配置trig模式

# 触发模式
trigModeDict = {}
trigModeDict["TRIG_force"] = 0
trigModeDict["TRIG_edge_up"] = 1
trigModeDict["TRIG_edge_down"] = 2
trigModeDict["TRIG_edge_both"] = 3
trigModeDict["TRIG_pulse_highLong"] = 4
trigModeDict["TRIG_pulse_highShort"] = 5
trigModeDict["TRIG_pulse_lowLong"] = 6
trigModeDict["TRIG_pulse_lowShort"] = 7

trigSrcNameList = ["CH1_Udc"]
# trigSrcNameList = ["ad2s1210ErrGPIO"]
trigSrcAddrList = [varDict[trigSrcNameX] for trigSrcNameX in trigSrcNameList]

# logTarNameList = ["CH1_Udc", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "targetTe", "omegaMfbk","thetaRDC_raw"]
# logTarNameList = ["CH1_Udc", "CH1_ifbk.V", "CH1_ifbk.W", "CH2_ifbk.U", "CH2_ifbk.V"]
# logTarNameList = ["CH1_Udc", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "CH2_Id", "CH2_Iq"]
logTarNameList = ["CH1_Udc", "CH1_ifbk.U", "omegaMfbk", "thetaRDC_raw"]

logTarAddrList = [varDict[logTarNameX] for logTarNameX in logTarNameList]
logTarCnt = len(logTarNameList)

buffSize = int(256 * 1024)

maxRecTick = int(buffSize/logTarCnt)

logKeepRatio = 0.2
logKeepDataNumBeforeTrig = int(logKeepRatio * maxRecTick)*logTarCnt

trigTimeTar = 40 * 2

# trigThd = 260
trigThd = 280
# trigThd = 205
# trigThd = 0.5

# 源模式
# bit 0-2 类型: 0 int16; 1 uint16; 2 int32; 3 uint32; 4 float; 5 强制0;
# bit 3 是否取反
# src[0]在低位 src[1]在高位

trigSrc = 0b0101_0100

# trigSrc=0b0101_0001

# 间隔采样
sampInter = 0

# trigMode = trigModeDict["TRIG_edge_down"]
# trigMode=trigModeDict["TRIG_edge_up"]
trigMode=trigModeDict["TRIG_force"]


In [ ]:
# 通过can写入配置

ch = canlib.openChannel(
    channel=0,
    flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
    bitrate=cdbBitrate,
)
ch.setBusOutputControl(canlib.Driver.NORMAL)
ch.busOn()

# 源地址
for ii in range(len(trigSrcAddrList)):
    txFrameData = struct.pack("<I", ii) + struct.pack("<I", trigSrcAddrList[ii])
    txFrame = Frame(id_=2, data=txFrameData, dlc=8)
    ch.write(txFrame)
    time.sleep(0.1)
ch.writeSync(timeout=500)

# 记录项
for ii in range(len(logTarAddrList)):
    txFrameData = struct.pack("<I", ii + 2) + struct.pack("<I", logTarAddrList[ii])
    txFrame = Frame(id_=2, data=txFrameData, dlc=8)
    ch.write(txFrame)
    time.sleep(0.1)
ch.writeSync(timeout=500)

# 需记录的数据个数 18
txFrameData = struct.pack("<I", 18) + struct.pack("<I", logTarCnt)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)
# 需保存的触发前的数据个数 19
txFrameData = struct.pack("<I", 19) + struct.pack("<I", logKeepDataNumBeforeTrig)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)
# 和时间有关触发的目标值 20
txFrameData = struct.pack("<I", 20) + struct.pack("<I", trigTimeTar)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)
# 触发阈值 21
txFrameData = struct.pack("<I", 21) + struct.pack("<f", trigThd)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)
# 触发值选择 22
txFrameData = struct.pack("<I", 22) + struct.pack("<I", trigSrc)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)
# 采样间隔 23
txFrameData = struct.pack("<I", 23) + struct.pack("<I", sampInter)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)
# 触发模式 24
txFrameData = struct.pack("<I", 24) + struct.pack("<I", trigMode)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)
time.sleep(0.1)

ch.writeSync(timeout=500)

# 触发器状态 25
txFrameData = struct.pack("<I", 25) + struct.pack("<I", 1)
txFrame = Frame(id_=2, data=txFrameData, dlc=8)
ch.write(txFrame)

ch.writeSync(timeout=500)

ch.busOff()
ch.close()

In [ ]:
# 特殊异常关端口
# Inactivate the CAN chip.
ch.busOff()

# Close the channel.
ch.close()